## 6.4.1 Summarising locations
We can already begin to plot the geo coordinates on a map to get an idea of how the tweets are distributed geographically. First we import the folium library, which contains the tools needed for generating and visualising geo location data. 

In [ ]:
import pandas as pd
import simplejson as json

df = pd.read_csv('./data/twitter_data_olio_UK_b.csv')
df.head()

## Point map

Next, we create a map on which we can populate our locations with the folium .Map() method. 

We then want to select the lat, long, and text columns and concatenate them together using the Python zip() function. You can think of this method as "zipping up" the data columns for use.

We next iterate through our selected dataset row-by-row, and create a folium CircleMarker object. 

The CirleMarker takes several arguments for passing the data to be visualised and our preferred style and format.  We can create the marker by calling the folium method CircleMarker, like so: folium.CircleMarker().  

We then call the .add_to() method on the return value on every iteration to add the marker to the map.

In [ ]:
import folium
from folium import plugins

tweet_map = folium.Map([51.507222, -0.1275], tiles='Stamen Terrain', zoom_start=6)

for lat, lng, text in zip(df.lat, df.lng, df.text): 
    folium.CircleMarker(
        [float(lat), float(lng)], # <- our latitude and longitude coordinates
        radius=3,
        color='green',
        fill=True,
        popup = text, # <- Our tweet text
        fill_color='darkgreen',
        fill_opacity=0.6
    ).add_to(tweet_map)

# Display the map
tweet_map


You can now explore the map by dragging the map and zooming in and out.  If you click on a single location you will see the tweet message as a popup.  This is a good start, we can see that there are more items being listed in major cities, such as Manchester, Birmingham and London, which is to be expected given the population numbers and transport links available in these regions. 

One issue we have, however, is that there are multiple items listed for the same location. At the moment we are only visualising a single location and item, any further data points are obscured.  To correct this we will cluster the items by the same location.

In [ ]:
tweet_map = folium.Map([51.507222, -0.1275], tiles='Stamen Terrain', zoom_start=5)

marker_cluster = plugins.MarkerCluster().add_to(tweet_map)

for lat, lng, text in zip(df.lat, df.lng, df.text): 
    folium.CircleMarker(
        [float(lat), float(lng)], # <- our latitude and longitude coordinates
        radius=3,
        color='green',
        fill=True,
        popup = text, # <- Our tweet text
        fill_color='darkgreen',
        fill_opacity=0.6
    ).add_to(marker_cluster)

# Display the map
tweet_map

Once again, drag and zoom to adjust the view. You will notice that as we zoom into a specific cluster, at some point it will split into the subgroups. This allows us to easily explore clusters of data points and corrects for the previous issue where points are layered on top of each other. Now when we view a single location, the items listed are all displayed as a series of connected nodes, which can be explored individually by clicking on them to view the popup.

We can also explore the tweets along the dimensions of time and space, by generating a heat map with a time element attached. This will enable us to see when and where the tweets were sent according to the hours of the day.

Folium provides a mehod for this, but we first need to convert our date to include an hour component. Twitter returns the date and time the tweet was created as a date string. We need to convert the values in the date column to a date object that we can query to extract the hour the tweet was sent.

Next we will iterate through the rows of our data set and pull out the latitude, longitude and hour from our date object. We group the latitude and longitude according to the hour using a Python dictionary. We then construct an index of the hours to use for plotting.

## Heatmap

In [ ]:
# Make sure our date columns contains date objects rather than a text string.
df['date'] = pd.to_datetime(df['date']) 

hour_list = {}
for lat, lng, hour in zip(df.lat,df.lng, df.date.dt.hour):
    hour_list.setdefault(hour,[]).append([lat,lng]) 

# index containing the hours extracted from the date and timestamp
index = [str(i) +':00' for i in range(len(hour_list.keys()))]

We will use a folium plugin HeatMapWithTime to generate a heatmap with our time element to view distribution of tweets sent over time and location using the index, and dictionary of coordinates grouped by hour. A small player will appear at the bottom of the plot, where you can click 'play' to view the evolution of the tweets according to the time of day.  

In [ ]:
tweet_map = folium.Map([51.507222, -0.1275], tiles='Stamen Terrain', zoom_start=6)

plugins.HeatMapWithTime(hour_list, index).add_to(tweet_map)

# Display the map
tweet_map

As expected we see very little activity during the early hours of the morning (around 03:00), but later on from about 07:00 we observe more tweets being sent out, before tweeting begins to drop off as the day draws to an end.  

Now that we know how to count up the number of tweets per location, we can start building more complex visualisations. We will build a choropleth map, which is a type of thematic map where areas are shaded in proportion to a statistical variable, such as  an aggregate summary over population density or income.  

The first step is to aggregate the data, we will aggregate over the town and city names, as well as at the county-level.  The tweets themselves do not include information about the county, however, as we learnt earlier we can easily extract this information using the reverse geo-coding approach and pulling out the relevant value from the data returned by geopy.

Due to the time it can take to retrieve data from an API when you have several thousand rows in your dataset, we have completed this step for you and prepared an updated dataset with county level informtation included.  

For this next activity we will load this updated data set, group the data according to towns, cities, and counties and then visualise the results using the folium.Choropleth() method. 

## Choropleth maps

In [ ]:
# Load the updated data set
df = pd.read_csv('./data/twitter_data_olio_UK_c.csv')
df.head()

In [ ]:
import numpy as np
# Load the updated data set
df = pd.read_csv('./data/twitter_data_olio_UK_c.csv')

# Summarise the locations by counting the place names in this case counties
df2 = df.groupby('place_county').size()

# select the column we need and call it 'count'
summary = pd.DataFrame(df2, columns=['count'])

# default index, otherwise 'groupby' column becomes the index
summary.reset_index(inplace=True) 

# Sort the values highest to smallest.
summary.sort_values(by='count', inplace=True, ascending=False)

# Convert the counts to proportions for plotting
summary['log'] = np.log2(summary['count'])

# Because the range of counts is skewed, where London has the greatest value
# We will convert the raw counts to log values for scaling.
total = np.log2(summary['count'].sum())

# Convert the resulting value to a percentage/proportion
summary['%'] = summary['log']/total

summary.head()

We now have our summary statistics on which to construct our choropleth map. We convert each count in our summary to a percentage to determine the degree of intensity of the shading. We still need one more piece of information in the form of boundary lines defined by the geographical region in the real world, so that we can clearly define the area that is being aggregated, that is we want to draw an outline around each location according to official geographic regions.

For this purpose we will use another data source that records this information. GeoJSON is a data standard designed for representing geographical features, based on the JSON format. Each location is stored as features including points (therefore addresses and locations, lines representing streets, and boundaries, and polygons for defining regions such as countries, and provinces. We have prepared a GeoJSON file with all the boundary lines for the locations in our dataset. Let's load this and inspect it a bit closer:

In [ ]:
# We load GeoJSON data at the county level.
with open('./data/county.geojson') as f:
    geo_data_str = f.read()    
geo_data = json.loads(geo_data_str)


Each row of our GeoJSON stores an entry for a location, together with features describing textual information and geolocation data defining points on a polygon that describe the area, let's look at the first row of our GeoJSON to get an idea of the structure:

In [ ]:
geo_data['features'][0] # View the first item in our GeoJSON file.

As you can see, we have some metadata about the location with a name property, which is followed by a long list of latitude and longitude coordinates stored in a list under the 'coordinates' attribute.  This represents the points necessary for defining a polygon that accurately represents the bounds of the region.  You can think of this array as a series of points like those found in a dot-to-dot illustration, where we draw a line from one point to another in sequential order until we recover the illustration composed of all these points and lines.

In [ ]:
# Create the Choropleth map based on our county summary counts
tweet_map = folium.Map([51.507222, -0.1275], zoom_start=6)

folium.Choropleth(
    geo_data=geo_data,
    name='choropleth',
    data=summary,
    columns=['place_county', '%'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.8,
    line_opacity=0.4,
    legend_name='Tweets (%)'
).add_to(tweet_map)

folium.LayerControl().add_to(tweet_map)

tweet_map

From the plot, you will learn that most of the tweets are centered around the east of England, with London and neighbouring boroughs being the highest tweeters in the data.